In [303]:
import pandas as pd
import numpy as np

In [304]:
df = pd.read_csv("31_Loc_state/ECN_2012_US_31SA1_with_ann.csv")

In [305]:
df.shape

(174956, 9)

In [306]:
print(df.iloc[0:4,])

                       GEO.id GEO.id2     GEO.display-label         NAICS.id  \
0  Geographic identifier code     Id2  Geographic area name  2012 NAICS code   
1                 0400000US01      01               Alabama            31-33   
2                 0400000US01      01               Alabama            31-33   
3                 0400000US01      01               Alabama            31-33   

          NAICS.display-label                         EMPSZES.id  \
0  Meaning of 2012 NAICS code  Employment size of establishments   
1               Manufacturing                                001   
2               Manufacturing                                219   
3               Manufacturing                                220   

                          EMPSZES.display-label YEAR.id  \
0  Meaning of Employment size of establishments    Year   
1                            All establishments    2012   
2          Establishments with 0 to 4 employees    2012   
3          Establishme

In [307]:
#the spreadsheet contains numbers of "establishments" (businesses)
# in the column ESTAB
#these are grouped by: 
# state (GEO.id2 or GEO.display-label)
# area  (NAICS.id or NAICS.display-label)
# number of employees (EMPSZES.id or EMPSZES.display-label)

In [308]:
#the spreadsheet has a weird structure where the header gives column names,
# and then the second row gives additional information, and the third and following are data...
#need to get rid of the second row
#also get rid of some redundant columns


In [309]:
df = df.iloc[1:,]
df = df.drop("GEO.id", axis=1)
df = df.drop("GEO.id2", axis=1)
df = df.drop("NAICS.id",axis=1)
df = df.drop("YEAR.id", axis=1)
df = df.drop("EMPSZES.display-label", axis=1)
df = df.rename(columns={"GEO.display-label":"State",\
                        "NAICS.display-label":"NAICS_Sector",\
                        "EMPSZES.id":"Size_Code",\
                        "ESTAB":"N_Establishments"})

In [310]:
print(df.iloc[0:4,])

     State   NAICS_Sector Size_Code N_Establishments
1  Alabama  Manufacturing       001             4286
2  Alabama  Manufacturing       219             1695
3  Alabama  Manufacturing       220              534
4  Alabama  Manufacturing       230              559


In [311]:
df["N_Establishments"] = df["N_Establishments"].astype(float)
df["Size_Code"] = df["Size_Code"].astype(int)
#size code is an object (string, at least in the first rows)
#but we need to convert it into an int!
#because string comparison will fail for a lot of the rows later in the document
#(maybe because they include non-printable chars  ?)

In [312]:
#now estimate number of employees
#first map size code to mean number of employees
#where we have converted size code to an integer
    
get_mean = lambda low,high : (high*(high + 1)/2.0 - (low-1)*low/2.0)/(high-low + 1)
    
size_code_to_mean_employee_number ={\
            1:0,\
            219:get_mean(0,4),\
            220:get_mean(5,9),\
            230:get_mean(10,19),\
            241:get_mean(20,49),\
            242:get_mean(50,99),\
            251:get_mean(100,249),\
            252:get_mean(250,499),\
            254:get_mean(500,999),\
            261:get_mean(1000,2499),\
            270:2500
            }


In [313]:
df["N_Employees"] = df["N_Establishments"]*df["Size_Code"].map(size_code_to_mean_employee_number)


df.loc[np.logical_not(df["Size_Code"]==1),"N_Establishments"] = 0
#set things up so that the groupby sum will give the correct number of establishments
#by getting rid of all counts except for total count

In [314]:
print(df.iloc[0:10,])

      State   NAICS_Sector  Size_Code  N_Establishments  N_Employees
1   Alabama  Manufacturing          1            4286.0          0.0
2   Alabama  Manufacturing        219               0.0       3390.0
3   Alabama  Manufacturing        220               0.0       3738.0
4   Alabama  Manufacturing        230               0.0       8105.5
5   Alabama  Manufacturing        241               0.0      21045.0
6   Alabama  Manufacturing        242               0.0      26373.0
7   Alabama  Manufacturing        251               0.0      57236.0
8   Alabama  Manufacturing        252               0.0      44940.0
9   Alabama  Manufacturing        254               0.0      48717.5
10  Alabama  Manufacturing        261               0.0      29741.5


In [315]:
df = df.drop("Size_Code", axis=1)

In [316]:
#now we just need to sum grouped by state and NAICS sector
#and we will have the total number of establishments
#and the estimated total number of employees

df_sum = df.groupby(by=["State","NAICS_Sector"], axis=0,sort=False).sum().reset_index()

In [317]:
print(df_sum.iloc[0:4,])

     State                     NAICS_Sector  N_Establishments  N_Employees
0  Alabama                    Manufacturing            4286.0     253286.5
1  Alabama               Food manufacturing             263.0      35983.5
2  Alabama   Dog and cat food manufacturing               4.0        525.5
3  Alabama  Other animal food manufacturing              31.0        794.5


In [318]:
#now we want to aggregate some of the sectors so they are more meaningful

In [319]:
print(np.unique(df_sum['NAICS_Sector']))

['Abrasive product manufacturing' 'Adhesive manufacturing'
 'Air and gas compressor manufacturing'
 'Air-conditioning and warm air heating equipment and commercial and industrial refrigeration equipment manufacturing'
 'Aircraft engine and engine parts manufacturing' 'Aircraft manufacturing'
 'All other basic organic chemical manufacturing'
 'All other converted paper product manufacturing'
 'All other leather good and allied product manufacturing'
 'All other miscellaneous chemical product and preparation manufacturing'
 'All other miscellaneous electrical equipment and component manufacturing'
 'All other miscellaneous fabricated metal product manufacturing'
 'All other miscellaneous food manufacturing'
 'All other miscellaneous general purpose machinery manufacturing'
 'All other miscellaneous manufacturing'
 'All other miscellaneous nonmetallic mineral product manufacturing'
 'All other miscellaneous textile product mills'
 'All other miscellaneous wood product manufacturing'
 'All

In [320]:
sector_terms = ["Aircraft", "Aluminum", "Chocolate",\
                "Paper", "Petro", "Plastics", "Semiconductor", "Textile", "Tobacco", "Vehicle", "Wood"]
#apply in_sector as a mask to all counts
for s in sector_terms:
    in_sector = 1.0*np.array([(s.lower() in name.lower()) for name in df_sum['NAICS_Sector']])
    df_sum[s+"_Employees"] = df_sum["N_Employees"]*in_sector
    df_sum[s+"_Establishments"] = df_sum["N_Establishments"]*in_sector

df_sum = df_sum.rename(columns={"N_Employees":"Total_Employees",\
                        "N_Establishments":"Total_Establishments"})
    
#sum again: sum over all of the disparate sectors captured by the individual terms   
df_sum = df_sum.groupby(by=["State"], axis=0,sort=False).sum().reset_index()


In [321]:
print(df_sum.iloc[0:4,])

      State  Total_Establishments  Total_Employees  Aircraft_Employees  \
0   Alabama               12858.0         759859.5              4044.5   
1    Alaska                1596.0          40069.5                37.0   
2   Arizona               12828.0         405990.0              9465.5   
3  Arkansas                8070.0         515649.0              3218.5   

   Aircraft_Establishments  Aluminum_Employees  Aluminum_Establishments  \
0                     21.0              3564.5                     36.0   
1                      6.0                11.0                      3.0   
2                     71.0              2349.5                     31.0   
3                     18.0               920.5                     19.0   

   Chocolate_Employees  Chocolate_Establishments  Paper_Employees  ...  \
0                 34.0                      12.0          22829.5  ...   
1                 54.5                       6.0             18.0  ...   
2                111.5         

In [322]:
pop_sizes = pd.read_csv("US_census_2012_state_population.csv")
pop_sizes["Population_2012"] = pop_sizes["Population_2012"].astype('float')

In [323]:
print(pop_sizes.iloc[0:4,])

      State  Population_2012
0   Alabama        4822023.0
1    Alaska         731449.0
2   Arizona        6553255.0
3  Arkansas        2949131.0


In [324]:
#set State name to be index for both dataframes
#to avoid any errors/bugs when we perform operations that involve both
pop_sizes.index = pop_sizes["State"]
pop_sizes = pop_sizes.drop("State", axis=1)
df_sum.index = df_sum["State"]
df_sum = df_sum.drop("State", axis=1)

In [328]:
print(pop_sizes.iloc[0:4,])
print(df_sum.iloc[0:4,])

          Population_2012
State                    
Alabama         4822023.0
Alaska           731449.0
Arizona         6553255.0
Arkansas        2949131.0
          Total_Establishments  Total_Employees  Aircraft_Employees  \
State                                                                 
Alabama                12858.0         759859.5              4044.5   
Alaska                  1596.0          40069.5                37.0   
Arizona                12828.0         405990.0              9465.5   
Arkansas                8070.0         515649.0              3218.5   

          Aircraft_Establishments  Aluminum_Employees  \
State                                                   
Alabama                      21.0              3564.5   
Alaska                        6.0                11.0   
Arizona                      71.0              2349.5   
Arkansas                     18.0               920.5   

          Aluminum_Establishments  Chocolate_Employees  \
State           

In [331]:
for c in df_sum.columns:
    df_sum = df_sum[c]/pop_sizes["Population_2012"]

In [333]:
print(df_sum.iloc[0:4,])

          Total_Establishments  Total_Employees  Aircraft_Employees  \
State                                                                 
Alabama               0.002667         0.157581            0.000839   
Alaska                0.002182         0.054781            0.000051   
Arizona               0.001958         0.061952            0.001444   
Arkansas              0.002736         0.174848            0.001091   

          Aircraft_Establishments  Aluminum_Employees  \
State                                                   
Alabama                  0.000004            0.000739   
Alaska                   0.000008            0.000015   
Arizona                  0.000011            0.000359   
Arkansas                 0.000006            0.000312   

          Aluminum_Establishments  Chocolate_Employees  \
State                                                    
Alabama                  0.000007             0.000007   
Alaska                   0.000004             0.000075  

In [334]:
df_sum = df_sum*10**4
#express everything in units of "per 10000 residents"

In [335]:
print(df_sum.iloc[0:4,])

          Total_Establishments  Total_Employees  Aircraft_Employees  \
State                                                                 
Alabama              26.665157      1575.810609            8.387558   
Alaska               21.819703       547.809895            0.505845   
Arizona              19.575005       619.524191           14.443967   
Arkansas             27.363993      1748.477772           10.913384   

          Aircraft_Establishments  Aluminum_Employees  \
State                                                   
Alabama                  0.043550            7.392126   
Alaska                   0.082029            0.150386   
Arizona                  0.108343            3.585241   
Arkansas                 0.061035            3.121258   

          Aluminum_Establishments  Chocolate_Employees  \
State                                                    
Alabama                  0.074657             0.070510   
Alaska                   0.041014             0.745096  

In [336]:
df_sum.to_csv("US_Census_2012_manufacturing_per_10000_residents.csv")